## Output toTFRecord
****
### possible examples
[tf tutorials example](https://www.tensorflow.org/tutorials/load_data/tf_records) <br>
[mnist to tfrecords](https://docs.azuredatabricks.net/_static/notebooks/deep-learning/mnist-tensorflow-to-tfrecords.html) <br>
[]() <br>
[]() <br>

In [1]:
import os
import time
import glob
import math
import multiprocessing as mp
import itertools
import hashlib
import tempfile

import numpy as np
import openslide
from skimage.filters import threshold_otsu

import PIL

In [2]:
output_dir = '../../DigiPath_MLTK_data/out_to_test'
if os.path.isdir(output_dir) == False:
    os.makedirs(output_dir)
    
OUTPUT_PATH = output_dir
data_dir = '../../DigiPath_MLTK_data/Aperio'
files_list = ['CMU-1-Small-Region.svs', 'CMU-1.svs']
SVS = os.path.join(data_dir, files_list[0])
print('Working with file:', SVS, '\nOutput path:', OUTPUT_PATH)

out_dir = '../../DigiPath_MLTK_data/out_to_test'
file_types = ['.jpg']
files_list = []
for maybe_file in os.listdir(out_dir):
    full_path = os.path.join(out_dir, maybe_file)
    if os.path.isfile(full_path):
        _, f_ext = os.path.splitext(maybe_file)
        if f_ext in file_types:
            files_list.append(full_path)

print(len(files_list))

Working with file: ../../DigiPath_MLTK_data/Aperio/CMU-1-Small-Region.svs 
Output path: ../../DigiPath_MLTK_data/out_to_test
0


In [3]:
def get_adjcent_segmented_length_fence_array(segment_length, length):
    fences = {}
    n_fenced = length // segment_length
    n_remain = 1 + length % segment_length
    paddit = n_remain // 2
    if n_remain == segment_length:
        paddit = 0
        n_remain = 0
        n_fenced = n_fenced + 1
    
    fence_array = np.zeros((n_fenced, 2)).astype(int)
    for k in range(n_fenced):
        if k == 0:
            fence_array[k, 0] = paddit
            fence_array[k, 1] = fence_array[k, 0] + segment_length - 1
        elif fence_array[k-1, 1] + segment_length <= length:
            fence_array[k, 0] = fence_array[k-1, 1] + 1
            fence_array[k, 1] = fence_array[k, 0] + segment_length - 1
        
    fences['fence_array'] = fence_array
    fences['n_fenced'] = n_fenced
    fences['n_remain'] = n_remain
    fences['padding'] = paddit

    return fences

segment_length = 224
length = segment_length*4 - 3

print('\tget_adjcent_segmented_length_fence_array')
print('\tn pixels: %i patch_length: %i\n'%(length, segment_length))
fences_dict = get_adjcent_segmented_length_fence_array(segment_length, length)
for k, v in fences_dict.items():
    if isinstance(v, np.ndarray):
        print('%20s:'%(k))
        for v_idx in range(v.shape[0]):
            if v[v_idx,1] - v[v_idx,0] != segment_length - 1:
                print('%24s, %6i   Bug Bug'%(v[v_idx,0], v[v_idx,1]))
            else:
                print('%24s, %6i'%(v[v_idx,0], v[v_idx,1]))
    else:
        print('%20s: %s'%(k,v))


	get_adjcent_segmented_length_fence_array
	n pixels: 893 patch_length: 224

         fence_array:
                     111,    334
                     335,    558
                     559,    782
            n_fenced: 3
            n_remain: 222
             padding: 111
